In [1]:
%matplotlib ipympl
import numpy as np
from pathlib import Path
import utils as utils
import harp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import aeon.io.video as video
from ipywidgets import widgets
from IPython.display import display
import re
import os
import zoneinfo
from datetime import datetime, timezone
import datetime
import matplotlib.pyplot as plt

In [3]:
# Load data
root = Path('/Volumes/harris/hypnose/rawdata/sub-027_id-068/ses-01_date-20250409/behav/2025-04-09T07-45-42')

In [ ]:
## Load relevant data streams

behavior_reader = harp.reader.create_reader('device_schemas/behavior.yml', epoch=harp.io.REFERENCE_EPOCH)
olfactometer_reader = harp.reader.create_reader('device_schemas/olfactometer.yml', epoch=harp.io.REFERENCE_EPOCH)

digital_input_data = utils.load(behavior_reader.DigitalInputState, root/"Behavior")
output_set = utils.load(behavior_reader.OutputSet, root/"Behavior")
output_clear = utils.load(behavior_reader.OutputClear, root/"Behavior")
olfactometer_valves_0 = utils.load(olfactometer_reader.OdorValveState, root/"Olfactometer0")
olfactometer_valves_1 = utils.load(olfactometer_reader.OdorValveState, root/"Olfactometer1")
olfactometer_end_0 = utils.load(olfactometer_reader.EndValveState, root/"Olfactometer0")
analog_data = utils.load(behavior_reader.AnalogData, root/"Behavior")
flow_meter = utils.load(olfactometer_reader.Flowmeter, root/"Olfactometer0")
heartbeat = utils.load(behavior_reader.TimestampSeconds, root/"Behavior")

# 'other' events
pulse_supply_1 = utils.load(behavior_reader.PulseSupplyPort1, root/"Behavior") # reward A pump end time
pulse_supply_2 = utils.load(behavior_reader.PulseSupplyPort2, root/"Behavior") # reward B pump end time
pulse_enable = utils.load(behavior_reader.OutputPulseEnable, root/"Behavior") # allow for either pump to deliver reward


In [ ]:
## Convert the time column which is an index into the first column of each event dataframe

for df in [
    heartbeat,
    digital_input_data,
    output_set,
    output_clear,
    olfactometer_valves_0,
    olfactometer_valves_1,
    olfactometer_end_0,
    analog_data,
    flow_meter,
    pulse_supply_1,
    pulse_supply_2,
    pulse_enable
]:
    df.reset_index(inplace=True) # Convert the index ("Time") into a column

In [6]:
## Convert all data streams from hardware timestamps to real timestamps

# Parse real-time reference from path (e.g., "2025-03-25T15-53-02")
real_time_str = root.as_posix().split('/')[-1]
# Parse the string as UTC and add timezone info
real_time_ref_utc = datetime.datetime.strptime(real_time_str, '%Y-%m-%dT%H-%M-%S').replace(tzinfo=timezone.utc)
# Convert UTC time to UK time (Europe/London, which handles DST automatically)
uk_tz = zoneinfo.ZoneInfo("Europe/London")
real_time_ref = real_time_ref_utc.astimezone(uk_tz)

# Determine hardware start time and compute offset
start_time_hardware = heartbeat['Time'].iloc[0]  # A Pandas Timestamp
start_time_dt = start_time_hardware.to_pydatetime()  # Now a Python datetime

# If the hardware timestamp is naive, assume it is in UK time
if start_time_dt.tzinfo is None:
    start_time_dt = start_time_dt.replace(tzinfo=uk_tz)

# Finally, compute the real-time offset
real_time_offset = real_time_ref - start_time_dt

# Create absolute (real-time) dataframes
digital_input_data_abs = digital_input_data.copy()
output_set_abs = output_set.copy()
output_clear_abs = output_clear.copy()
olfactometer_valves_0_abs = olfactometer_valves_0.copy()
olfactometer_valves_1_abs = olfactometer_valves_1.copy()
olfactometer_end_0_abs = olfactometer_end_0.copy()
analog_data_abs = analog_data.copy()
flow_meter_abs = flow_meter.copy()
pulse_supply_1_abs = pulse_supply_1.copy()
pulse_supply_2_abs = pulse_supply_2.copy()
pulse_enable_abs = pulse_enable.copy()

for df_abs in [
    digital_input_data_abs,
    output_set_abs,
    output_clear_abs,
    olfactometer_valves_0_abs,
    olfactometer_valves_1_abs,
    olfactometer_end_0_abs,
    analog_data_abs,
    flow_meter_abs,
    pulse_supply_1_abs,
    pulse_supply_2_abs,
    pulse_enable_abs
]:
    df_abs['Time'] = df_abs['Time'] + real_time_offset

In [7]:
## Extract odour port event timings for initiation and exiting

# Extract all DIPort0 events
port0_events = digital_input_data_abs['DIPort0']

# Build a DataFrame for poke initiation events (TRUE)
init_poke = port0_events[port0_events == True].to_frame(name='EventValue')
init_poke['Time'] = digital_input_data_abs.loc[init_poke.index, 'Time'].values
init_poke = init_poke[['Time', 'EventValue']]  # keep only "Time" and event value

# Build a list for exit events, then convert to a DataFrame
init_poke_exit_list = []
for idx in init_poke.index:
    subsequent_false = port0_events[(port0_events.index > idx) & (port0_events == False)]
    if not subsequent_false.empty:
        next_false_idx = subsequent_false.index[0]
        next_false_time = digital_input_data_abs.loc[next_false_idx, 'Time']
        init_poke_exit_list.append({'Time': next_false_time, 'EventValue': False})

init_poke_exit = pd.DataFrame(init_poke_exit_list)

In [8]:
## Extract relevant events for reward pokes and delivery from the *abs dataframes and store 'Time' in each variable

# Port activation events
r1_poke = digital_input_data_abs[digital_input_data_abs['DIPort1'] == True].copy()
r1_poke = r1_poke[['Time', 'DIPort1']]  # Keep only Time and DIPort1

r2_poke = digital_input_data_abs[digital_input_data_abs['DIPort2'] == True].copy()
r2_poke = r2_poke[['Time', 'DIPort2']]

# Reward events
r1_reward = pulse_supply_1_abs[['Time']].copy()
r1_reward['PulseSupplyPort1'] = True

r2_reward = pulse_supply_2_abs[['Time']].copy()
r2_reward['PulseSupplyPort2'] = True

# Olfactometer valve events
r1_olf_valve = olfactometer_valves_0_abs[olfactometer_valves_0_abs['Valve0'] == True].copy()
r1_olf_valve = r1_olf_valve[['Time', 'Valve0']]

r2_olf_valve = olfactometer_valves_0_abs[olfactometer_valves_0_abs['Valve1'] == True].copy()
r2_olf_valve = r2_olf_valve[['Time', 'Valve1']]

In [14]:
## Insert all events into a single DataFrame

def create_unique_series(events_df):
    """Creates a unique-timestamp boolean series (event = True) from a DataFrame that has a Time column."""
    timestamps = events_df['Time']
    # Handle duplicates by adding microsecond offsets
    if len(timestamps) != len(set(timestamps)):
        unique_timestamps = []
        seen = set()
        for ts in timestamps:
            counter = 0
            ts_modified = ts
            while ts_modified in seen:
                counter += 1
                ts_modified = ts + pd.Timedelta(microseconds=counter)
            seen.add(ts_modified)
            unique_timestamps.append(ts_modified)
        timestamps = unique_timestamps
    return pd.Series(True, index=timestamps)

# Example: create Series from various *abs dataframes (already in real-time)
# IMPORTANT: Ensure these DataFrames (init_poke, r1_poke, r2_poke, r1_reward, r2_reward,
# r1_olf_valve, r2_olf_valve) and real_time_ref are defined beforehand.
init_poke_series    = create_unique_series(init_poke)
r1_poke_series      = create_unique_series(r1_poke)
r2_poke_series      = create_unique_series(r2_poke)
r1_reward_series    = create_unique_series(r1_reward)
r2_reward_series    = create_unique_series(r2_reward)
r1_olf_valve_series = create_unique_series(r1_olf_valve)
r2_olf_valve_series = create_unique_series(r2_olf_valve)

# Combine into a single DataFrame
events_df = pd.DataFrame({
    'init_poke':    init_poke_series,
    'r1_poke':      r1_poke_series,
    'r2_poke':      r2_poke_series,
    'r1_reward':    r1_reward_series,
    'r2_reward':    r2_reward_series,
    'r1_olf_valve': r1_olf_valve_series,
    'r2_olf_valve': r2_olf_valve_series
})

# Convert the index (real-time Timestamps) to a column
events_df = events_df.reset_index(names='timestamp')

# Convert 'timestamp' column to datetime and localize tz-naive timestamps to UK time
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'], errors='coerce')
events_df['timestamp'] = events_df['timestamp'].apply(
    lambda ts: ts if ts.tzinfo is not None else ts.replace(tzinfo=uk_tz)
)

# Drop rows with invalid timestamps (if any)
events_df = events_df.dropna(subset=['timestamp'])

# Fill missing event columns with False and sort
events_df = events_df.fillna(False).sort_values('timestamp')

# Mark start_time from real_time_ref (assumed to be tz-aware in UK time)
events_df['start_time'] = False
start_time = real_time_ref  # use your existing real_time_ref here

start_time_row = pd.DataFrame({
    'timestamp': [start_time],
    'init_poke': [False],
    'r1_poke': [False],
    'r2_poke': [False],
    'r1_reward': [False],
    'r2_reward': [False],
    'r1_olf_valve': [False],
    'r2_olf_valve': [False],
    'start_time': [True]
})

# Ensure start_time_row has the same column order as events_df
start_time_row = start_time_row[events_df.columns]

# Concatenate start_time_row with events_df and sort
events_df_adjusted = pd.concat([start_time_row, events_df], ignore_index=True).sort_values('timestamp')

# Remove timezone info from the 'timestamp' column so it displays as naive datetime
events_df_adjusted['timestamp'] = events_df_adjusted['timestamp'].dt.tz_localize(None)

# Display the adjusted DataFrame
display(events_df_adjusted)

/var/folders/_n/p_b9638x5g99dbmz8kz99s100000gn/T/ipykernel_7842/881542611.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events_df = events_df.fillna(False).sort_values('timestamp')


,timestamp,init_poke,r1_poke,r2_poke,r1_reward,r2_reward,r1_olf_valve,r2_olf_valve,start_time
0,2025-04-09 08:45:42.000000000,False,False,False,False,False,False,False,True
1,2025-04-09 08:48:20.149760000,False,False,True,False,False,False,False,False
2,2025-04-09 08:48:21.450336000,False,False,True,False,False,False,False,False
3,2025-04-09 08:48:50.786272000,False,False,True,False,False,False,False,False
4,2025-04-09 08:49:31.546176000,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
1178,2025-04-09 10:49:33.837600000,False,True,False,False,False,False,False,False
1179,2025-04-09 10:49:33.994720000,False,True,False,False,False,False,False,False
1180,2025-04-09 10:49:39.500287999,False,False,True,False,False,False,False,False
1181,2025-04-09 10:49:39.512480000,False,False,False,False,True,False,False,False


In [15]:
## Determine number of rewards delivered and input volume of each reward to determine total volume delivered

# Define the input volume per reward (e.g., in microliters)
reward_volume_r1 = 8.0  # Volume per r1 (Reward A) delivery
reward_volume_r2 = 8.0  # Volume per r2 (Reward B) delivery

# Determine number of reward events from the reward dataframes
num_r1_rewards = r1_reward.shape[0]
num_r2_rewards = r2_reward.shape[0]

# Calculate total volume delivered for each reward port
total_volume_r1 = num_r1_rewards * reward_volume_r1
total_volume_r2 = num_r2_rewards * reward_volume_r2

# Sum to get overall total volume delivered
total_volume_delivered = total_volume_r1 + total_volume_r2

print(f"Number of Reward A (r1) delivered: {num_r1_rewards} (Total Volume: {total_volume_r1})")
print(f"Number of Reward B (r2) delivered: {num_r2_rewards} (Total Volume: {total_volume_r2})")
print(f"Overall total volume delivered: {total_volume_delivered}")

Number of Reward A (r1) delivered: 89 (Total Volume: 712.0)
Number of Reward B (r2) delivered: 90 (Total Volume: 720.0)
Overall total volume delivered: 1432.0


In [16]:
## Determine session length

# Calculate session duration in seconds
start_time_sec = heartbeat['TimestampSeconds'].iloc[0]
end_time_sec = heartbeat['TimestampSeconds'].iloc[-1]
session_duration_sec = end_time_sec - start_time_sec

# Convert to hours, minutes, seconds
hours = int(session_duration_sec // 3600)
minutes = int((session_duration_sec % 3600) // 60)
seconds = int(session_duration_sec % 60)

print(f"Session duration: {hours} hours, {minutes} minutes, {seconds} seconds")

Session duration: 2 hours, 3 minutes, 58 seconds
